<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train10_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [67]:

train10 = pd.read_csv(r'/content/train_data10.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [68]:

train10['Target'] = train10['Target'].replace(['Others'],'Invalid')


In [69]:
train10['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [70]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train10_Xtfidf = tfidf_vect.fit_transform(train10['Sentence'])


In [72]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [73]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [74]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [75]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [76]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [77]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [78]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train10_Xtfidf,train10['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6736725663716814
                  precision    recall  f1-score   support

        Analysis       0.55      0.39      0.46       131
      Conclusion       0.66      0.47      0.55        45
           Facts       0.75      0.91      0.82       485
         Invalid       0.47      0.50      0.49       123
           Issue       0.58      0.42      0.48        53
Rule/Law/Holding       0.67      0.21      0.32        67

        accuracy                           0.67       904
       macro avg       0.61      0.48      0.52       904
    weighted avg       0.66      0.67      0.65       904



In [79]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [80]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 3]
[0.9863183698193295, 0.9934010330816322, 0.9999371668158571, 0.9991984975221606, 0.9922015057831927, 0.9949729521502737, 0.9948688086023355, 0.9974384419391232, 0.9980981828851114, 0.9995687998636024, 0.9996515636631754, 0.9802996582262528, 0.99039445565572, 0.9974756537721051, 0.9963387944706799, 0.9977645829457594, 0.9932766122522938, 0.9994590620770177, 0.9996482105366152, 0.9883422677904372, 0.994944455579278, 0.9957512116773679, 0.9921831921721628, 0.9951885338041274, 0.9997431405412153, 0.993250643592875, 0.9877056900235259, 0.9998639958033355, 0.9947304015163397, 0.981875218076022, 0.9948082486728004, 0.9958598890177793, 0.9996197638285749, 0.9993443526958152, 0.9994139062379657, 0.9967733567028562]
[0, 4, 5, 11, 12, 17, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 41, 45, 48, 50, 52, 55, 56, 60, 62, 70, 72, 76, 78, 84, 87, 89, 94, 95, 96, 98]
36
36
36


In [81]:
unlabel_1=unlabel_1.loc[[0, 4, 5, 11, 12, 17, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 41, 45, 48, 50, 52, 55, 56, 60, 62, 70, 72, 76, 78, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts


In [82]:
frame_1 = [train10,unlabel_1]
train10_1 = pd.concat(frame_1)
len(train10_1)

3512

In [83]:
x_train_1 = tfidf_vect.transform(train10_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train10_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6657183499288762
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       102
      Conclusion       0.61      0.50      0.55        40
           Facts       0.72      0.91      0.80       351
         Invalid       0.56      0.42      0.48       113
           Issue       0.70      0.50      0.58        42
Rule/Law/Holding       0.58      0.33      0.42        55

        accuracy                           0.67       703
       macro avg       0.62      0.51      0.55       703
    weighted avg       0.65      0.67      0.64       703



In [84]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 5, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 5, 2]
[0.996791311384645, 0.9995259344082914, 0.9987681626697523, 0.9877612485270324, 0.9968324747518199, 0.9956495809973582, 0.9999442486134376, 0.9935430932498517, 0.9812306420019871, 0.9979299100357599, 0.9888155390280335, 0.9975037081274827, 0.988941819380679, 0.9914248682873449, 0.987975673687579, 0.9990358883754604, 0.9998460438375869, 0.9941279739718534, 0.9999391613052728, 0.9807862527822184, 0.9874854286403165, 0.9995551329499605, 0.9904196235960095, 0.9860152034352352, 0.9986655139606975, 0.9974216463041962, 0.9872551462186504, 0.9997002954465212, 0.9849360182561508, 0.9989149271597708, 0.999847980844861, 0.9973482176768685, 0.9992035195190951, 0.9986134248098363, 0.9948523698912469, 0.9968414622896732, 0.9837646377946347, 0.9956868011772935, 0.9998636975952933, 0.9934072330670095, 0.9845647613981935, 0.995306514366404]
[1, 2, 4, 5, 7, 8, 12, 13, 14, 20, 21

In [85]:
unlabel_2 = unlabel_2.loc[[1, 2, 4, 5, 7, 8, 12, 13, 14, 20, 21, 23, 24, 32, 34, 36, 37, 39, 42, 44, 48, 49, 50, 51, 54, 55, 58, 59, 60, 63, 64, 69, 70, 71, 75, 83, 85, 86, 88, 91, 93, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
5,2278,evidence state appellant shows appellant alleg...,Analysis
7,3150,finding door facing street seen man walking pr...,Facts


In [86]:
frame_2 = [train10_1,unlabel_2]
train10_2 = pd.concat(frame_2)
len(train10_2)

3554

In [88]:
x_train_2 = tfidf_vect.transform(train10_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train10_2['Target'],test_size=0.24,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6400937866354045
                  precision    recall  f1-score   support

        Analysis       0.46      0.47      0.46       119
      Conclusion       0.77      0.44      0.56        45
           Facts       0.69      0.90      0.78       421
         Invalid       0.57      0.40      0.47       146
           Issue       0.57      0.29      0.38        59
Rule/Law/Holding       0.67      0.25      0.37        63

        accuracy                           0.64       853
       macro avg       0.62      0.46      0.51       853
    weighted avg       0.63      0.64      0.61       853



In [89]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9985572970961747, 0.9943582482874808, 0.9933066607823284, 0.9976373568267894, 0.9864276134789383, 0.9997206856798139, 0.999800855344475, 0.9998669823874153, 0.9928687081655058, 0.9998396656616532, 0.9913539345748222, 0.9811025588100603, 0.9942867919113011, 0.9999248256529527, 0.9898299973826499, 0.997854458988769, 0.985126377856501, 0.9890000536416261, 0.9990250605845666, 0.9950756081160761, 0.999702935315611, 0.999235537106175, 0.9832587376421614, 0.9974172326794917, 0.9934865457442066, 0.9989107827522388, 0.9882531151900587, 0.9992612810644936, 0.9974888181688972, 0.9815175965621944, 0.9994489582875387, 0.9964331951410546, 0.9997855079368391, 0.9970201778321808, 0.9976915346141759, 0.9998431575536572, 0.9967387941147914, 0.9964553918556215, 0.9972615598701284, 0.9822772255201079, 0.9948804610963927, 0.9998432677810198, 0.9857739228496983]
[0, 2, 3, 4, 6

In [90]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 6, 7, 8, 12, 15, 20, 23, 24, 26, 27, 29, 30, 34, 35, 36, 40, 43, 46, 47, 49, 51, 52, 53, 54, 56, 62, 63, 64, 65, 66, 67, 68, 70, 72, 75, 78, 80, 89, 97],:]
frame_3 = [train10_2,unlabel_3]
train10_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train10_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train10_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6680555555555555
                  precision    recall  f1-score   support

        Analysis       0.44      0.39      0.41        96
      Conclusion       0.70      0.52      0.60        44
           Facts       0.76      0.89      0.82       386
         Invalid       0.51      0.44      0.47       109
           Issue       0.61      0.46      0.53        41
Rule/Law/Holding       0.40      0.23      0.29        44

        accuracy                           0.67       720
       macro avg       0.57      0.49      0.52       720
    weighted avg       0.65      0.67      0.65       720



In [91]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 5, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 3, 0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2]
[0.9992843484114304, 0.9875527469396752, 0.9982342487416553, 0.9937464050718477, 0.9998456785044537, 0.983326825365496, 0.9995497699272707, 0.9997156626343527, 0.9993432813565255, 0.9977739477939258, 0.9993149018937306, 0.9960026535990265, 0.9990472933693676, 0.9930043265146818, 0.9979517105821174, 0.9955317284302238, 0.998323444937091, 0.9983843232431269, 0.9989064581819964, 0.9998918546586484, 0.9937222636890913, 0.9919493546969571, 0.9939636695588395, 0.9950971050283539, 0.9979539736116201, 0.9977098150309436, 0.9965410602244144, 0.9836847205176906, 0.98840444567627, 0.9983025962369231, 0.9802307171975255, 0.9964705539097425, 0.9996729852080646, 0.9988437841467454, 0.9866526333708455, 0.9999767757336471, 0.9962484442364238, 0.998532699052716, 0.9853859940793439, 0.9984577727607137, 0.9835514308025831, 0.9975771633366151]
[2, 6, 7, 8, 10, 12, 15, 18, 21, 22, 

In [92]:
unlabel_4=unlabel_4.loc[[2, 6, 7, 8, 10, 12, 15, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 46, 49, 50, 55, 56, 58, 60, 61, 66, 67, 68, 69, 70, 71, 72, 75, 80, 87, 89, 93, 94, 99],:]

In [93]:
frame_4 = [train10_3,unlabel_4]
train10_4 = pd.concat(frame_4)
len(train10_4)
x_train_4 = tfidf_vect.transform(train10_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train10_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6744659206510681
                  precision    recall  f1-score   support

        Analysis       0.47      0.41      0.44       133
      Conclusion       0.62      0.64      0.63        45
           Facts       0.78      0.88      0.83       527
         Invalid       0.49      0.44      0.46       149
           Issue       0.58      0.42      0.49        62
Rule/Law/Holding       0.55      0.36      0.43        67

        accuracy                           0.67       983
       macro avg       0.58      0.53      0.55       983
    weighted avg       0.66      0.67      0.66       983



In [94]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 1, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 0]
[0.999594267285755, 0.99614092461623, 0.9926672622119603, 0.9913181307422894, 0.9929404719190275, 0.9990883021626048, 0.9969691791324714, 0.9994666232577788, 0.999728158380213, 0.9933897641194973, 0.9879736478429448, 0.9915586436924142, 0.9997871822747462, 0.9823359131999316, 0.9960955711293382, 0.9987615135117387, 0.9990113658047525, 0.9860523169830534, 0.999184550802442, 0.9902819758476544, 0.9942354922013588, 0.9959299084129319, 0.9975197162665699, 0.9959993443563376, 0.9996874626676815, 0.997444444832198, 0.995515400158249, 0.9987397293108083, 0.9991323102769981, 0.9899161994585098, 0.9896505795919697, 0.9846721513774888, 0.9975170726018013, 0.9951348463400747]
[3, 7, 10, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 42, 46, 52, 53, 56, 60, 63, 66, 67, 73, 76, 82, 86, 91, 94, 95, 97, 100, 104, 107, 110]
34
34
34


In [95]:
unlabel_5 = unlabel_5.loc[[3, 7, 10, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 42, 46, 52, 53, 56, 60, 63, 66, 67, 73, 76, 82, 86, 91, 94, 95, 97, 100, 104, 107, 110],:]

In [105]:
frame_5 = [train10_4,unlabel_5]
train10_5 = pd.concat(frame_5)
len(train10_5)
x_train_5 = tfidf_vect.transform(train10_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train10_5['Target'],test_size=0.24,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6870748299319728
                  precision    recall  f1-score   support

        Analysis       0.52      0.44      0.47       124
      Conclusion       0.58      0.58      0.58        43
           Facts       0.78      0.87      0.82       460
         Invalid       0.53      0.54      0.53       139
           Issue       0.68      0.56      0.62        50
Rule/Law/Holding       0.63      0.33      0.44        66

        accuracy                           0.69       882
       macro avg       0.62      0.55      0.58       882
    weighted avg       0.68      0.69      0.68       882



In [106]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))


Accuracy 0.6775431861804223


In [107]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6455579554445606, 0.5848955205256816, 0.6051469257123342, None)
